# Projeto 1 - módulo 6

## Precificação dinâmica - e-commerce

### Mercari Price Suggestion Challenge - Kaggle

Mercari é um site de revenda de produtos online. Uma dos desafios desse tipo de plataforma é auxiliar o usuário, muitas vezes com pouco conhecimento de vendas, a determinar um preço para os seus produtos de modo a maximizar as chances de venda.

### Sobre este projeto

O presente projeto tem o objetivo de desenvolver um algoritmo que identifique produtos já vendidos similares e sugira ao usuário um preço ótimo para novos produtos cadastrados.


### Preparação do ambiente

Para este projeto, acesse o link https://www.kaggle.com/competitions/mercari-price-suggestion-challenge/overview 


In [44]:
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score , mean_absolute_error as MAE, mean_squared_error

In [13]:
base_train = pd.read_csv ('/content/drive/MyDrive/Blue Edtech/data/processed/train_data.csv') 
base_test = pd.read_csv ('/content/drive/MyDrive/Blue Edtech/data/processed/test_data.csv') 

In [14]:
base_train

,name,category_1,category_2,category_3,item_condition_id,brand_name,price,shipping,date,stock,item_description
0,iphone 6s plus defender case,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins",1,No Brand,10.0,1,15-6-2018,2,size small euc excellent used condition kimchi...
1,beautiful rainbow eye iphone plus case,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins",1,No Brand,7.0,1,12-12-2018,1,new men s rbx performance boxer brief size lar...
2,hold under armour women s t shirt bundle,Women,Athletic Apparel,Shirts & Tops,2,Under Armour,20.0,1,13-5-2018,11,piece decor bundle beautiful rustic teal blue ...
3,nike shorts bundle,Women,Athletic Apparel,Shorts,3,Nike,18.0,1,24-2-2018,4,iphone kate spade case new never used was a gi...
4,bare mineral eye shadow lot,Beauty,Makeup,Eyes,3,No Brand,14.0,0,29-10-2018,11,silpada new necklace sterling silver pearl bra...
...,...,...,...,...,...,...,...,...,...,...,...
1037157,herschel backpack,Women,Women's Handbags,Backpack Style,2,No Brand,20.0,0,30-7-2018,14,brand new never worn gerber girl s size months...
1037158,illamasqua beyond powder in omg,Beauty,Makeup,Face,1,No Brand,25.0,1,19-8-2018,7,brand new high quality in stores blurs imperfe...
1037159,free ship tracking,Other,Office supplies,Shipping Supplies,1,No Brand,3.0,1,29-12-2018,7,no description yet
1037160,pink 3ds xl,Electronics,Video Games & Consoles,Consoles,3,Nintendo,85.0,1,23-3-2018,7,full size not used retail before taxes or fees...


In [15]:
base_train['texto'] = base_train['name'] + ' ' + base_train['category_1'] + ' ' + base_train['category_2'] + ' ' + base_train['category_3'] + ' ' + base_train['item_condition_id'].astype(str) + base_train['brand_name'] + ' ' + base_train['shipping'].astype(str) + ' ' + base_train['item_description']
base_test['texto'] = base_test['name'] + ' ' + base_test['category_1'] + ' ' + base_test['category_2'] + ' ' + base_test['category_3'] + ' ' + base_test['item_condition_id'].astype(str) + base_test['brand_name'] + ' ' + base_test['shipping'].astype(str) + ' ' + base_test['item_description']

In [16]:
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# extraindo características

textos_train = base_train[['texto']]
stop_words = stopwords.words('english')
tf = TfidfVectorizer(analyzer = 'word', min_df=0, stop_words = stop_words)
tfidf_matrix_train = tf.fit_transform(textos_train['texto'].values.astype('U'))
textos_test = base_test[['texto']]
tfidf_matrix_test = tf.transform(textos_test['texto'].values.astype('U'))
print(tfidf_matrix_train.shape)
tfidf_matrix_test.shape

(1037162, 171732)


(444499, 171732)

In [77]:
def ecommerce(produto):

  # calculando a similaridade de coseno do produto selecionado com os demais

  medida = cosine_similarity(tfidf_matrix_test[produto], tfidf_matrix_train)
  select = (base_test.index == produto)
  produto_cadastrado = base_test[select]

# criando a função que identifica o index dos produtos com maiores coeficiente de similaridade e impõe umm limite de 0.5 

  limite = 0.3
  result = []

  for cnt in np.arange(10):
    ind = cnt - 10 # ind vai de -10 a -1
    index = medida[0].argsort()[ind] # retornando os índices q ordenam o array
    measure = medida[0][index]

    # if index == produto:
    #   continue
    if (measure > limite):
      result.append([index,measure])

# criando listas com os indices e scores dos 9 produtos com as maiores similaridades em relação ao novo produto cadastrado

  indices = []
  scores = []

# resultado_parcial = recomendacoes(medida, produto) 
  for ind, scr in result:
   if ind not in indices:
     indices.append(ind)
     scores.append(scr)

# criando um datraframe com os produto identificados

  produtos_similares = pd.DataFrame()

  for ind in indices:
    select = (base_train.index == ind)
    df_temp = base_train[select]
    produtos_similares = pd.concat([produtos_similares, df_temp], axis = 0)

  produtos_similares['score'] = scores

  return produtos_similares

In [ ]:
# # Teste você mesmo!

# produto = int(input('Digite índice do produto: \n'))
# select = (base_test.index == produto)
# produto_cadastrado = base_test[select]
# produto_cadastrado[['name', 'category_1', 'category_2', 'category_3', 'item_description', 'item_condition_id', 'brand_name', 'price']]

In [ ]:
# # lista de produtos similares

# resultado = ecommerce(produto)
# resultado[['name', 'category_1', 'category_2', 'category_3', 'item_description', 'item_condition_id', 'brand_name', 'price']]

In [ ]:
# preco_min = resultado['price'].min()
# preco_max = resultado['price'].max()
# preco_medio = resultado['price'].mean()
# preco_moda = resultado['price'].mode()
# print(preco_min, preco_max, preco_medio, preco_moda)
# print(produto_cadastrado['price'])

In [87]:
preco_medio = pd.DataFrame(columns = ['price'])

for produto in range(0,200):
  resultado = ecommerce(produto)
  if len(resultado.index) > 0:
    valor = resultado['price'].mean()
    preco_medio.loc[produto] = [valor]
  else:
    preco_medio.loc[produto] = ['not found']

In [88]:
# adaptando o tamanho do teste para calcular as métricas

y_test = base_test.iloc[:200,:]

In [89]:
# calculando as métricas do modelo
print('Root Mean squared erro: %.2f' % np.sqrt(mean_squared_error(y_test['price'], preco_medio['price'])))
print('erro médio absoluto: %.2f' % MAE(y_test['price'], preco_medio['price']))
print('R score: %.2f' % r2_score(y_test['price'], preco_medio['price']))

Root Mean squared erro: 31.28
erro médio absoluto: 13.93
R score: 0.07
